In [45]:
import openai
import os
import requests
from PIL import Image
import dotenv

# import dotenv
dotenv.load_dotenv()
ev = dict(os.environ)
print(ev)

# Get endpoint and key from environment variables
openai.api_key = os.getenv("AZURE_OPENAI_KEY","").strip()
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT","").strip()

# Assign the API version (DALL-E is currently supported for the 2023-06-01-preview API version only)
openai.api_version = '2023-10-01-preview'
openai.api_type = 'azure'

{'__CFBundleIdentifier': 'com.apple.Terminal', 'TMPDIR': '/var/folders/7r/ybz255g54cs8yq9zhswm2qcw0000gn/T/', 'XPC_FLAGS': '0x0', 'TERM': 'xterm-color', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.uzajjJKDrX/Listeners', 'XPC_SERVICE_NAME': '0', 'TERM_PROGRAM': 'Apple_Terminal', 'TERM_PROGRAM_VERSION': '452', 'TERM_SESSION_ID': '191D7B03-B704-4E0C-BFA3-6A76AA0EAD9A', 'SHELL': '/bin/zsh', 'HOME': '/Users/I559573', 'LOGNAME': 'I559573', 'USER': 'I559573', 'PATH': '/Library/Frameworks/Python.framework/Versions/3.10/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/I559573/Library/Python/3.8/bin:/opt/homebrew/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin:/Applications/Privileges.app/Contents/Resources', 'SHLVL': '1', 'PWD': '/Users/I559573/Documents/GitHub/generative-ai-for-begi

In [46]:
import time
import json
import httpx
import openai


class CustomHTTPTransport(httpx.HTTPTransport):
    def handle_request(
        self,
        request: httpx.Request,
    ) -> httpx.Response:
        if "images/generations" in request.url.path and request.url.params[
            "api-version"
        ] in [
            "2023-06-01-preview",
            "2023-07-01-preview",
            "2023-08-01-preview",
            "2023-09-01-preview",
            "2023-10-01-preview",
        ]:
            request.url = request.url.copy_with(path="/openai/images/generations:submit")
            response = super().handle_request(request)
            operation_location_url = response.headers["operation-location"]
            request.url = httpx.URL(operation_location_url)
            request.method = "GET"
            response = super().handle_request(request)
            response.read()

            timeout_secs: int = 120
            start_time = time.time()
            while response.json()["status"] not in ["succeeded", "failed"]:
                if time.time() - start_time > timeout_secs:
                    timeout = {"error": {"code": "Timeout", "message": "Operation polling timed out."}}
                    return httpx.Response(
                        status_code=400,
                        headers=response.headers,
                        content=json.dumps(timeout).encode("utf-8"),
                        request=request,
                    )

                time.sleep(int(response.headers.get("retry-after")) or 10)
                response = super().handle_request(request)
                response.read()

            if response.json()["status"] == "failed":
                error_data = response.json()
                return httpx.Response(
                    status_code=400,
                    headers=response.headers,
                    content=json.dumps(error_data).encode("utf-8"),
                    request=request,
                )

            result = response.json()["result"]
            return httpx.Response(
                status_code=200,
                headers=response.headers,
                content=json.dumps(result).encode("utf-8"),
                request=request,
            )
        return super().handle_request(request)


client = openai.AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key=os.getenv("AZURE_OPENAI_KEY","").strip(),
    api_version="2023-10-01-preview",
    http_client=httpx.Client(
        transport=CustomHTTPTransport(),
    ),
)

In [68]:
disallow_list = f"""swords, violence, blood, gore, nudity, sexual content, adult content, adult themes, 
adult language, adult humor, adult jokes, adult situations, adult"""

metaprompt = f"""

You are an assistant designer that creates images for children. 

The image needs to be in color.  

The image needs to be in landscape orientation.  

The image needs to be in a 16:9 aspect ratio.
"""

prompt = f"""
Generate monument of the Arc of Triumph in Paris, France, in the evening light with a small child 
holding a Teddy looks on.
"""

In [69]:
print(prompt)


Generate monument of the Arc of Triumph in Paris, France, in the evening light with a small child 
holding a Teddy looks on.



In [70]:
try:
    # Create an image by using the image generation API
    generation_response = client.images.generate(
        # Enter your prompt text here
        #prompt='Bunny on horse, holding a lollipop, on a foggy meadow where it grows daffodils',
        prompt=prompt,
        size='1024x1024',
        n=1,
        #temperature=0,
    )
    # Set the directory for the stored image
    image_dir = os.path.join(os.curdir, 'images')

    # If the directory doesn't exist, create it
    if not os.path.isdir(image_dir):
        os.mkdir(image_dir)

    # Initialize the image path (note the filetype should be png)
    image_path = os.path.join(image_dir, 'generated-image.png')

    # Retrieve the generated image
    image_url = generation_response.data[0].url  # extract image URL from response
    generated_image = requests.get(image_url).content  # download the image
    with open(image_path, "wb") as image_file:
        image_file.write(generated_image)

    # Display the image in the default image viewer
    image = Image.open(image_path)
    image.show()

# catch exceptions
except openai.APIError as err:
    print(err)

In [ ]:
response = client.images.edit(
    model="dall-e-2",
    image=open("image_edit_original.webp", "rb"),
    mask=open("image_edit_mask.webp", "rb"),
    prompt="A sunlit indoor lounge area with a pool containing a flamingo",
    n=1,
    size="1024x1024"
)

image_url = response.data[0].url

In [ ]:
response = client.images.create_variation(
  image=open("generated-image.png", "rb"),
  n=1,
  size="1024x1024"
)

image_url = response.data[0].url

In [72]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

BadRequestError: Error code: 400 - {'error': {'message': 'Invalid content type. image_url is only supported by certain models.', 'type': 'invalid_request_error', 'param': 'messages.[0].content.[1].type', 'code': None}}

In [73]:
import base64
import requests

# OpenAI API Key
api_key = "YOUR_OPENAI_API_KEY"

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "generated-image.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What’s in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

NameError: name 'api_key' is not defined